In [ ]:
# git フォルダごと削除用
import shutil
import os
shutil.rmtree('./wikiextractor/')

日本語wikiからコーパスを作成するスクリプトです.<br>
https://dumps.wikimedia.org/jawiki/latest/ <br>
こちらのサイトから最新版の"pages-articles"のアドレスを手に入れてください. <br>

In [ ]:
# MeCabのインストール
!pip install mecab-python3
!pip install unidic-lite

In [ ]:
! wget https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-articles.xml.bz2

--2021-08-04 12:37:24--  https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3429329342 (3.2G) [application/octet-stream]
Saving to: ‘jawiki-latest-pages-articles.xml.bz2’

jawiki-latest-pages 100%[===================>]   3.19G  2.89MB/s    in 12m 1s  

2021-08-04 12:49:26 (4.53 MB/s) - ‘jawiki-latest-pages-articles.xml.bz2’ saved [3429329342/3429329342]



ダンプデータには不要なマークアップなどが含まれているので、取り除くためのテキストクリーニング用のスクリプトをgitから持ってきます

In [ ]:
# pip でwikiextractorはインストールする
!pip install wikiextractor
#! git clone https://github.com/attardi/wikiextractor.git

     |████████████████████████████████| 46 kB 3.1 MB/s 


日本語wikiに対してテキストクリーニングを実行します

In [ ]:
!python -m wikiextractor.WikiExtractor -o extracted jawiki-latest-pages-articles.xml.bz2

INFO: Preprocessing 'jawiki-latest-pages-articles.xml.bz2' to collect template definitions: this may take some time.
INFO: Preprocessed 100000 pages
INFO: Preprocessed 200000 pages
INFO: Preprocessed 300000 pages
INFO: Preprocessed 400000 pages
INFO: Preprocessed 500000 pages
INFO: Preprocessed 600000 pages
INFO: Preprocessed 700000 pages
INFO: Preprocessed 800000 pages
INFO: Preprocessed 900000 pages
INFO: Preprocessed 1000000 pages
INFO: Preprocessed 1100000 pages
INFO: Preprocessed 1200000 pages
INFO: Preprocessed 1300000 pages
INFO: Preprocessed 1400000 pages
INFO: Preprocessed 1500000 pages
INFO: Preprocessed 1600000 pages
INFO: Preprocessed 1700000 pages
INFO: Preprocessed 1800000 pages
INFO: Preprocessed 1900000 pages
INFO: Preprocessed 2000000 pages
INFO: Preprocessed 2100000 pages
INFO: Preprocessed 2200000 pages
INFO: Preprocessed 2300000 pages
INFO: Preprocessed 2400000 pages
INFO: Preprocessed 2500000 pages
INFO: Preprocessed 2600000 pages
INFO: Loaded 92221 templates in 11

テキストに前処理を加えた上で,複数のtxtファイルをひとつに結合します

In [ ]:
import glob
from bs4 import BeautifulSoup

with open('./tmp.txt','w') as f:
    for directory in glob.glob('./extracted/*'):
        for name in glob.glob(directory+'/*'):
            with open(name, 'r') as r:
                for line in r:
                    # titleを削除する
                    if '<doc ' in line:
                        next(r)
                        next(r)
                    elif '</doc>' in line:
                        f.write('\n')
                        continue
                    else:
                        # 空白・改行削除、大文字を小文字に変換
                        text = BeautifulSoup(line.strip()).text.lower()
                        f.write(text)

/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.e-vic.co.jp/jp_contents/column/index.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.e-vic.co.jp/jp_contents/house/index.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.imdb.com/name/nm3676269/?ref_=fn_al_nm_1" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behin

ここからはBERTのトレーニング用にテキストファイルを整形していきます.<br>
文章を単語ごとに分割し, ひとつの単元の中に偶数個の文章が含まれるように調整します.

In [ ]:
import linecache
import random
import MeCab

In [ ]:
random.seed(42)
filename = 'tmp.txt'
save_file = 'even_rows1G.txt'
LIMIT_BYTE = 1000000000 # 1Gbyte
#save_file = 'even_rows100M.txt'
#LIMIT_BYTE = 100000000 # 100Mbyte
t = MeCab.Tagger('-Owakati')

def get_byte_num(s):
    return len(s.encode('utf-8'))

In [ ]:
with open(save_file, 'w') as f:
    count_byte = 0
    with open(filename) as r:
        for text in r:
            print('{} bytes'.format(count_byte))
            text = t.parse(text).strip()
            # 一文ごとに分割する
            text = text.split('。')
            # 空白要素は捨てる
            text = [t.strip() for t in text if t]
            # 一単元の文書が偶数個の文章から成るようにする(BERTのデータセットの都合上)
            max_text_len = len(text) // 2
            text = text[:max_text_len * 2]
            text = '\n'.join(text)
            f.write(text)
            count_byte += get_byte_num(text)
            if count_byte >= LIMIT_BYTE:
                break

ストリーミング出力は最後の 5000 行に切り捨てられました。
991428958 bytes
991432592 bytes
991434060 bytes
991435693 bytes
991436330 bytes
991436646 bytes
991441557 bytes
991441557 bytes
991442759 bytes
991449085 bytes
991449760 bytes
991450203 bytes
991451485 bytes
991451485 bytes
991452423 bytes
991456193 bytes
991456193 bytes
991456193 bytes
991456193 bytes
991456193 bytes
991457289 bytes
991462005 bytes
991462005 bytes
991463138 bytes
991466442 bytes
991466442 bytes
991466442 bytes
991466769 bytes
991466769 bytes
991470883 bytes
991470883 bytes
991473124 bytes
991473382 bytes
991473593 bytes
991474778 bytes
991475733 bytes
991476229 bytes
991477313 bytes
991479781 bytes
991491080 bytes
991491080 bytes
991491080 bytes
991492481 bytes
991492481 bytes
991492481 bytes
991492481 bytes
991492481 bytes
991504336 bytes
991528795 bytes
991528795 bytes
991528795 bytes
991530270 bytes
991539232 bytes
991539232 bytes
991539232 bytes
991539232 bytes
991539232 bytes
991539719 bytes
991539719 bytes
991540014 bytes
99154257

これでBERTの学習に使うデータセットができました.<br>
今度はTraining用とValidation用のデータに分割します.

In [ ]:
num_lines = sum(1 for line in open(save_file))
print('Base file lines : ', num_lines)
# 全体の80%をTraining dataに当てます
train_lines = int(num_lines * 0.8)
print('Train file lines : ', train_lines)

Base file lines :  5370024
Train file lines :  4296019


dataは前処理済みテキスト保存場所 <br>
outputは訓練モデル保存場所として作成

In [ ]:
! mkdir -p data output

In [ ]:
out_file_name_temp = './data/splitted_%d.txt'

split_index = 1
line_index = 1
out_file = open(out_file_name_temp % (split_index,), 'w')
in_file = open(save_file)
line = in_file.readline()
while line:
    if line_index > train_lines:
        print('Starting file: %d' % split_index)
        out_file.close()
        split_index = split_index + 1
        line_index = 1
        out_file = open(out_file_name_temp % (split_index,), 'w')
    out_file.write(line)
    line_index = line_index + 1
    line = in_file.readline()
    
out_file.close()
in_file.close()

Starting file: 1


In [ ]:
print('Train file lines : ', sum(1 for line in open('./data/splitted_1.txt')))
print('Valid file lines : ', sum(1 for line in open('./data/splitted_2.txt')))

Train file lines :  4296019
Valid file lines :  1074005


これにてテキストの前処理は完了です！